In [59]:
import random
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab

In [60]:
# here, coin number is saved as float (complying with code from github, while contradicts white paper)
class Smartcoin(object):
	def __init__(self, name='kennycoin', reservetokenName='aCoin', initCRR=0.5, initPrice=1, initIssueNum=100):
		self._Name = name
		self._ReservetokenName = reservetokenName
		self._CRR = float(initCRR)
		self._Price = float(initPrice)
		self._Supply = float(initIssueNum)
		self._ReserveBalance = float(initCRR * initIssueNum)
		self._budget = float(0)     

	def printInfo(self):
		print '---------'
		print 'NAME:', self._Name, '| RESERVE NAME:', self._ReservetokenName, 'CRR:', self._CRR
		print 'PRICE:',self._Price
		print 'SUPPLY:', self._Supply, '| RESERVE BALANCE:', self._ReserveBalance
		print 'BUDGET:', self._budget
        
	def updatePrice(self, reserveBalance, supply, CRR):
		newPrice = reserveBalance/(supply * CRR)
		return newPrice
    
	def setCRR(self, newCRR = 0.5):
		oldCRR = self._CRR
		self._CRR = newCRR
		print 'CRR', oldCRR, '->', newCRR

	def getPrice(self):
		return self._Price

	def purchasing(self, convertIntoNum=0):
		# ETH be convert into BNT
		issuedtokenNum = self._Supply * (((self._ReserveBalance + convertIntoNum)/self._ReserveBalance)**(self._CRR) - 1)
		self._Supply = self._Supply + issuedtokenNum
		self._ReserveBalance = self._ReserveBalance + convertIntoNum
		oldPrice = self._Price
		self._Price = self.updatePrice(self._ReserveBalance, self._Supply, self._CRR)
		increaseRatio = (self._Price - oldPrice)/oldPrice
		print '*********'
		print round(convertIntoNum,2), self._ReservetokenName+" be converted into as", round(issuedtokenNum,2), self._Name
		print "Current prize of "+self._Name+" is", self._Price, "with", round(increaseRatio*100,4), "% increasing."
		self._budget = self._budget + self._Price*self._Supply - oldPrice*(self._Supply - issuedtokenNum)
		return issuedtokenNum

	def destroying(self, convertOutNum=0):
		# BNT be converted out to ETH
		destroyedtokenNum = convertOutNum      
		reserveReceivedNum = self._ReserveBalance*(1 - ((self._Supply - convertOutNum)/self._Supply)**(1/self._CRR))
		self._Supply = self._Supply - destroyedtokenNum
		self._ReserveBalance = self._ReserveBalance - reserveReceivedNum
		oldPrice = self._Price
		self._Price = self.updatePrice(self._ReserveBalance, self._Supply, self._CRR)
		decreaseRatio = (oldPrice-self._Price)/oldPrice
		print '*********'
		print round(convertOutNum,2), self._Name+" be converted out as", round(reserveReceivedNum,2), self._ReservetokenName
		print "Current prize of "+self._Name+" is", self._Price, "with", round(decreaseRatio*100,4), "% decreasing."
		self._budget = self._budget + self._Price*self._Supply - oldPrice*(self._Supply + destroyedtokenNum)
		return reserveReceivedNum

In [61]:
class Customers(object):
    def __init__(self, smartToken, smartValue = 0, reserveValue = 500):
        # _smartToken is the token customers want to buy -- SmartToken()
        self._smartToken = smartToken
        # _ownedSmartToken refers to smart token's number customers have
        self._ownedSmartToken = float(0)
        # _smartValue is the value of ownedSmartTokens -- currentPrize * owned#
        self._smartValue = float(0)
        self._reserveValue = float(reserveValue)
        # _ownedvalue = _reserveValue + _smartValue
        self._ownedvalue = float(reserveValue)
        # gain or lose money comparing to the original state
        self._budget = float(0)
        
    def printinfo(self):
        # since new reserve will be converted into or out the kenny coin, the budget could change
        # when print info, update the budget
        oldownedValue = self._ownedvalue
        self._ownedvalue = self._smartToken._Price * self._ownedSmartToken + self._reserveValue
        self._budget = self._budget + self._ownedvalue - oldownedValue
        print '------'
        print 'smartToken Name:', self._smartToken._Name
        print 'ownedvalue:', self._ownedvalue, 'budget:', self._budget
        
    def getReserveValue(self):
        return self._reserveValue
        
    def purchase(self, reserveTokenNumber):
        oldPrice = self._smartToken._Price
        if reserveTokenNumber > self._reserveValue:
            print 'Invalid Operation in Purchase'
            return
        # smartToken.purchasing means being converted
        issuedSmartToken = self._smartToken.purchasing(reserveTokenNumber)
        newPrice = self._smartToken._Price      
        self._ownedSmartToken = self._ownedSmartToken + issuedSmartToken
        self._smartValue = self._ownedSmartToken * newPrice
        self._reserveValue = self._reserveValue - reserveTokenNumber
        # in fact, in this case ownedValue and budget actually does not change at all
        oldownedValue = self._ownedvalue
        self._ownedvalue = self._reserveValue + self._smartValue
        self._budget = self._budget + self._ownedvalue - oldownedValue
        
        
    def destroy(self, smartTokenNumber):
        oldPrice = self._smartToken._Price
        if self._ownedSmartToken < smartTokenNumber:
            print 'Invalid Operation in Destroy'
            return
        self._ownedSmartToken = self._ownedSmartToken - smartTokenNumber
        # smartToken.purchasing means being purchased
        ReceivedToken = self._smartToken.destroying(smartTokenNumber)
        newPrice = self._smartToken._Price
        self._smartValue = self._ownedSmartToken*newPrice
        self._reserveValue = self._reserveValue + ReceivedToken
        # In fact, in this case ownedValue actually does not change at all
        oldownedValue = self._ownedvalue
        self._ownedvalue = self._reserveValue + self._smartValue
        self._budget = self._budget + self._ownedvalue - oldownedValue
        

In [64]:
KennyCoin = Smartcoin(name='Kenny',reservetokenName='ETH',initCRR=0.2, initPrice=1,initIssueNum=300000)

In [66]:
Alice = Customers(smartToken=KennyCoin)
Alice.purchase(100)
Alice.printinfo()
Alice.destroy(99.93)
Alice.printinfo()

*********
100.0 ETH be converted into as 99.93 Kenny
Current prize of Kenny is 1.00134555251 with 0.1333 % increasing.
------
smartToken Name: Kenny
ownedvalue: 500.066621227 budget: 0.0666212272207
*********
99.93 Kenny be converted out as 100.0 ETH
Current prize of Kenny is 1.00001247415 with 0.1331 % decreasing.
------
smartToken Name: Kenny
ownedvalue: 500.0 budget: 3.45039552485e-11


In [13]:
custNum = 1000
custOriginalReserve = 500

In [14]:
# create a customer list
custlist = []
i = 0
while i < custNum:
    custitem = Customers(smartToken = KennyCoin, reserveValue = custOriginalReserve)
    custlist.append(custitem)
    i = i + 1

In [36]:
while i <= 100:
    # uses seed to let the purchase or sell happen randomly by a certain ratio
    seed = random.randint(1, 100) 
    # assume Joe is the holder of Kennycoin
    Joe = custlist[i]
    if seed <= 50:
        # Joe is purchasing
        custReserve = Joe.getReserveValue()
        randomBuy = random.uniform(0,custReserve)
        # Joe make randomBuyCoin(#) reserve coins convert into KennyCoin
        randomBuyCoin = randomBuy / KennyCoin.getPrice()
        Joe.purchase(TokenNumber = randomBuyCoin)
    else:
        # Joe is selling 
        
    i = i + 1

In [54]:
# example of using smartcoin class
BNTCoin = Smartcoin(name='BNT',reservetokenName='ETH',initCRR=0.2, initPrice=1,initIssueNum=300000)
BNTCoin.printInfo()
BNTCoin.purchasing(300)
BNTCoin.printInfo()
BNTCoin.purchasing(700)
BNTCoin.printInfo()
BNTCoin.destroying(1302)
BNTCoin.printInfo()
BNTCoin.purchasing(100)
BNTCoin.printInfo()

---------
NAME: BNT | RESERVE NAME: ETH CRR: 0.2
PRICE: 1.0
SUPPLY: 300000.0 | RESERVE BALANCE: 60000.0
BUDGET: 0.0
*********
300.0 ETH be converted into as 299.4 BNT
Current prize of BNT is 1.00399800399 with 0.3998 % increasing.
---------
NAME: BNT | RESERVE NAME: ETH CRR: 0.2
PRICE: 1.00399800399
SUPPLY: 300299.401794 | RESERVE BALANCE: 60300.0
BUDGET: 1500.0
*********
700.0 ETH be converted into as 694.0 BNT
Current prize of BNT is 1.01331125792 with 0.9276 % increasing.
---------
NAME: BNT | RESERVE NAME: ETH CRR: 0.2
PRICE: 1.01331125792
SUPPLY: 300993.399232 | RESERVE BALANCE: 61000.0
BUDGET: 5000.0
*********
1302.0 BNT be converted out as 1307.97 ETH
Current prize of BNT is 0.995891667702 with 1.7191 % decreasing.
---------
NAME: BNT | RESERVE NAME: ETH CRR: 0.2
PRICE: 0.995891667702
SUPPLY: 299691.399232 | RESERVE BALANCE: 59692.0334754
BUDGET: -1539.83262295
*********
100.0 ETH be converted into as 100.35 BNT
Current prize of BNT is 0.997226150558 with 0.134 % increasing.
---